In [12]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import ta

In [15]:
df = pd.read_csv(r'C:\Users\ABC\Downloads\SOCR-HeightWeight.csv')

In [14]:
df.head(7)

,Date,Open,High,Low,Close,Adj Close,Volume,Prev_close,change,Percentage_change,Gab,Gab_percentage,SMA5,SMA15,SMA30,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,2/7/2019,289.399994,294.000000,286.600006,287.399994,275.050049,19733315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/8/2019,285.600006,288.000000,282.200012,286.000000,273.710205,19781908,287.399994,-1.399994,-0.49%,-1.799988,-0.63%,283.250000,269.84,280.805001,-10.96,NaN,NaN
2,2/11/2019,285.950012,285.950012,280.000000,280.500000,268.446564,15420017,286.000000,-5.500000,-1.92%,-0.049988,-0.02%,277.949997,269.20,281.565000,-12.36,0,NaN
3,2/12/2019,280.649994,282.200012,274.649994,275.399994,263.565704,15326106,280.500000,-5.100006,-1.82%,0.149994,0.05%,271.250000,269.04,282.848334,-13.80,0,NaN
4,2/14/2019,268.000000,268.350006,263.649994,267.100006,255.622375,26372671,275.399994,-8.299988,-3.01%,-7.399994,-2.69%,265.025009,269.47,284.426667,-14.96,0,NaN
5,2/15/2019,268.850006,268.850006,261.299988,262.950012,251.650726,23366091,267.100006,-4.149994,-1.55%,1.750000,0.66%,261.450012,270.41,286.490000,-16.08,0,NaN
6,2/18/2019,264.399994,265.500000,258.799988,259.950012,248.779633,14781744,262.950012,-3.000000,-1.14%,1.449982,0.55%,261.425003,272.03,288.408333,-16.37,0,NaN


In [7]:


# Load your dataframe, assuming it's already loaded

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Input the number of days

n = int(input("Enter the number of days: "))

# Initialize lists to store aggregated data
dates = []
opens = []
highs = []
lows = []
closes = []

# Find the first available date in the dataset
start_date = df['Date'].iloc[0]

# Iterate until the end of the dataset
i = 0
while i < len(df):
    # Select data for the current n-day period
    filtered_data = df.iloc[i:i+n]
    
    # Aggregate data for the current period
    if not filtered_data.empty:
        dates.append(filtered_data['Date'].iloc[0])
        opens.append(filtered_data['Open'].iloc[0])
        highs.append(filtered_data['High'].max())
        lows.append(filtered_data['Low'].min())
        closes.append(filtered_data['Close'].iloc[-1])
    
    # Move to the next n-day period
    i += n

# Create candlestick chart using Plotly
fig = go.Figure(data=[go.Candlestick(x=dates,
                                     open=opens,
                                     high=highs,
                                     low=lows,
                                     close=closes)])
# Add moving averages
fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA5'], mode='lines', name='SMA5'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA15'], mode='lines', name='SMA15'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA30'], mode='lines', name='SMA30'))

# Add buy and sell signals


# Customize the layout
fig.update_layout(title=f'Candlestick Chart for TCS (From {start_date.strftime("%m/%d/%Y")} to 2024 for {n} days per period)',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show the plot
fig.show()


Enter the number of days: 1


In [10]:
import pandas as pd
import plotly.graph_objects as go
import ta

# Load your DataFrame
df = pd.read_csv(r'C:\Users\ABC\Downloads\SBI.csv')

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

n = int(input("Enter the number of days:"))

# Calculate SMAs
df['SMA15'] = ta.trend.sma_indicator(df['Close'], window=15)
df['SMA30'] = ta.trend.sma_indicator(df['Close'], window=30)

resampled_data = df.resample(f'{n}D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'SMA15': 'last',
    'SMA30': 'last'
}).dropna()

# Calculate the difference between SMA15 and SMA30
resampled_data['SMA_Difference'] = resampled_data['SMA15'] - resampled_data['SMA30']

# Add 'ENTER/EXIT' column
resampled_data['ENTER/EXIT'] = ''

# Find the points where SMA15-SMA30 transitions from positive to negative (SELL signal) or negative to positive (BUY signal)
sell_indices = resampled_data[(resampled_data['SMA_Difference'] < 0) & (resampled_data['SMA_Difference'].shift(-1) >= 0)].index
buy_indices = resampled_data[(resampled_data['SMA_Difference'] > 0) & (resampled_data['SMA_Difference'].shift(-1) <= 0)].index

# Update 'ENTER/EXIT' column
resampled_data.loc[sell_indices, 'ENTER/EXIT'] = 'EXIT'
resampled_data.loc[buy_indices, 'ENTER/EXIT'] = 'ENTER'

# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(x=resampled_data.index,
                                     open=resampled_data['Open'],
                                     high=resampled_data['High'],
                                     low=resampled_data['Low'],
                                     close=resampled_data['Close'])])

# Add traces for SMA15 and SMA30
fig.add_trace(go.Scatter(x=resampled_data.index, y=resampled_data['SMA15'], mode='lines', line=dict(color='yellow'), name='SMA15'))
fig.add_trace(go.Scatter(x=resampled_data.index, y=resampled_data['SMA30'], mode='lines', line=dict(color='purple'), name='SMA30'))

# Highlight ENTER/EXIT points
enter_indices = resampled_data[resampled_data['ENTER/EXIT'] == 'ENTER'].index
exit_indices = resampled_data[resampled_data['ENTER/EXIT'] == 'EXIT'].index

fig.add_trace(go.Scatter(x=enter_indices, y=resampled_data.loc[enter_indices, 'SMA30'],
                         mode='markers', name='EXIT', marker=dict(color='red', size=10)))
fig.add_trace(go.Scatter(x=exit_indices, y=resampled_data.loc[exit_indices, 'SMA30'],
                         mode='markers', name='ENTER', marker=dict(color='green', size=10)))

# Update layout
fig.update_layout(title=f'Candlestick Chart with SMA and Enter/Exit Signals for Every {n} Days of AdaniPower',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show the figure
fig.show()


Enter the number of days:1


In [9]:
pip install ta

Note: you may need to restart the kernel to use updated packages.
